In [3]:
import os
import shutil
import tempfile
%pip install hydromt_sfincs
import hydromt_sfincs
from pathlib import Path

import numpy as np
import xarray as xr
import rioxarray
import gcsfs

from hydromt_sfincs import SfincsModel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [hydromt_sfincs]m [hydromt]lltree]
Note: you may need to restart the kernel to use updated packages.


In [4]:
SCRATCH_BUCKET = os.environ.get("SCRATCH_BUCKET", "gs://leap-scratch/renriviera")
OUT_PREFIX = f"{SCRATCH_BUCKET.rstrip('/')}/sfincs_soundview_preproc"

# ---- Raster inputs (FINAL/CLEAN as you wrote) ----
PATHS_GCS = {
    "dep_dem":      f"{OUT_PREFIX}/rasters/final/dep_dem_utm25m_meters.tif",
    "landcover":    f"{OUT_PREFIX}/rasters/clean/landcover_worldcover_utm25m.tif",
    "manning":      f"{OUT_PREFIX}/rasters/final/manning_n_utm25m.tif",
    "impervious":   f"{OUT_PREFIX}/rasters/final/impervious_frac_utm25m.tif",
    "curve_number": f"{OUT_PREFIX}/rasters/final/curve_number_cn_utm25m.tif",
}

# ---- Wind forcing (your FEWS netamu/amv file) ----
WIND_GCS = f"{OUT_PREFIX}/forcing/wind/sfincs_netamuamv_hrrr_u10v10_soundview_2020.nc"

# ---- Precip forcing (teammate assembled) ----
# you said: gcs_path = f"{OUT_PREFIX}/forcing/sfincs.precip"
PRECIP_GCS = f"{OUT_PREFIX}/forcing/sfincs.precip"

print("OUT_PREFIX:", OUT_PREFIX)
print("DEM:", PATHS_GCS["dep_dem"])
print("Wind:", WIND_GCS)
print("Precip:", PRECIP_GCS)


OUT_PREFIX: gs://leap-scratch/renriviera/sfincs_soundview_preproc
DEM: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/dep_dem_utm25m_meters.tif
Wind: gs://leap-scratch/renriviera/sfincs_soundview_preproc/forcing/wind/sfincs_netamuamv_hrrr_u10v10_soundview_2020.nc
Precip: gs://leap-scratch/renriviera/sfincs_soundview_preproc/forcing/sfincs.precip


In [6]:
# ============================================================
# 3) Download model inputs locally (AUTHENTICATED gcsfs)
#    Fixes: 401 Anonymous caller error on gs://leap-scratch/*
# ============================================================

import tempfile
from pathlib import Path
import gcsfs

cache_dir = Path(tempfile.mkdtemp(prefix="sfincs_run_inputs_"))
print("Local input cache:", cache_dir)

def make_gcsfs():
    # Try authenticated modes first
    for tok in ["google_default", "cloud"]:
        try:
            fs = gcsfs.GCSFileSystem(token=tok)
            # tiny call to validate auth works
            _ = fs.ls("leap-scratch/renriviera", detail=False)[:3]
            print(f"✅ gcsfs authenticated with token='{tok}'")
            return fs
        except Exception as e:
            print(f"⚠️ token='{tok}' failed:", repr(e))

    # LAST resort: anonymous (will fail for private buckets)
    fs = gcsfs.GCSFileSystem(token="anon")
    print("⚠️ Falling back to anonymous gcsfs (will not work on leap-scratch)")
    return fs

fs_gcs = make_gcsfs()

def strip_gs(path: str) -> str:
    if path.startswith("gs://"):
        return path[len("gs://"):]
    return path

def gcs_exists(gs_path: str) -> bool:
    return fs_gcs.exists(strip_gs(gs_path))

def gcs_to_local(gs_path: str, local_path: Path, recursive: bool = False):
    g = strip_gs(gs_path)
    local_path.parent.mkdir(parents=True, exist_ok=True)

    if not fs_gcs.exists(g):
        raise FileNotFoundError(f"GCS path does not exist or not accessible: {gs_path}")

    # If "recursive" OR it's a prefix/folder-like dataset, download recursively
    fs_gcs.get(g, str(local_path), recursive=recursive)

    if not local_path.exists():
        raise FileNotFoundError(f"Download failed: {gs_path} -> {local_path}")

    return local_path

# ---- Download rasters ----
paths_local = {}
print("\n--- Download rasters locally ---")
for k, p in PATHS_GCS.items():
    out = cache_dir / Path(p).name
    print(f"{k:12s} exists={gcs_exists(p)} -> {p}")
    paths_local[k] = gcs_to_local(p, out, recursive=False)
    print(f"✅ {k:12s} -> {paths_local[k]}")

# ---- Download wind netcdf ----
print("\n--- Download wind forcing locally ---")
print("wind exists=", gcs_exists(WIND_GCS), "->", WIND_GCS)
wind_local = gcs_to_local(WIND_GCS, cache_dir / "netamuamvfile.nc", recursive=False)
print("✅ wind ->", wind_local)

# ---- Download precip (file OR folder) ----
print("\n--- Download precip locally ---")
precip_local = cache_dir / "sfincs_precip"
precip_path_clean = strip_gs(PRECIP_GCS)

if fs_gcs.exists(precip_path_clean):
    # Try file download first
    try:
        precip_local_file = cache_dir / "sfincs_precip"
        fs_gcs.get(precip_path_clean, str(precip_local_file), recursive=False)
        if precip_local_file.exists():
            precip_local = precip_local_file
            print("✅ precip file ->", precip_local)
        else:
            raise RuntimeError("Precip file download returned nothing.")
    except Exception as e:
        print("⚠️ precip is likely a folder/prefix, switching to recursive download:", repr(e))
        precip_local.mkdir(parents=True, exist_ok=True)
        fs_gcs.get(precip_path_clean, str(precip_local), recursive=True)
        print("✅ precip folder ->", precip_local)
else:
    raise FileNotFoundError(f"❌ Precip path not found or not accessible: {PRECIP_GCS}")

print("\n✅ All inputs cached locally in:", cache_dir)


Local input cache: /tmp/sfincs_run_inputs_1mk6ynd6
✅ gcsfs authenticated with token='google_default'

--- Download rasters locally ---
dep_dem      exists=True -> gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/dep_dem_utm25m_meters.tif
✅ dep_dem      -> /tmp/sfincs_run_inputs_1mk6ynd6/dep_dem_utm25m_meters.tif
landcover    exists=True -> gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/clean/landcover_worldcover_utm25m.tif
✅ landcover    -> /tmp/sfincs_run_inputs_1mk6ynd6/landcover_worldcover_utm25m.tif
manning      exists=True -> gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/manning_n_utm25m.tif
✅ manning      -> /tmp/sfincs_run_inputs_1mk6ynd6/manning_n_utm25m.tif
impervious   exists=True -> gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/impervious_frac_utm25m.tif
✅ impervious   -> /tmp/sfincs_run_inputs_1mk6ynd6/impervious_frac_utm25m.tif
curve_number exists=True -> gs://leap-scratch/renriviera/sfincs_sound

In [12]:
# ============================================================
# 4) Initialize SFINCS region + grid (ROBUST FIX)
#    ✅ Setup region from DEM bbox (EPSG:26918)
#    ✅ FORCE grid to match FINAL DEM exactly (no setup_grid_from_region)
# ============================================================

import numpy as np
import geopandas as gpd
from shapely.geometry import box
import xarray as xr

# ------------------------------------------------------------
# 0) DEM must exist (loaded earlier). If not, load it.
# ------------------------------------------------------------
try:
    _ = dem.rio.crs
except Exception:
    raise RuntimeError(
        "❌ `dem` is not defined in this notebook scope.\n"
        "Make sure you loaded the FINAL DEM meters before this cell, e.g.:\n"
        "dem = rioxarray.open_rasterio(paths_local['dep_dem']).squeeze()"
    )

# --- bbox from DEM (UTM meters) ---
minx, miny, maxx, maxy = map(float, dem.rio.bounds())
region_bbox = [minx, miny, maxx, maxy]

crs = dem.rio.crs
res = abs(float(dem.rio.resolution()[0]))

print("DEM bounds:", region_bbox)
print("DEM CRS:", crs)
print("DEM res:", res)

# ------------------------------------------------------------
# 1) Build region geometry IN THE DEM CRS (EPSG:26918)
# ------------------------------------------------------------
region_poly = box(minx, miny, maxx, maxy)
region_gdf = gpd.GeoDataFrame(
    {"name": ["soundview_bbox"]},
    geometry=[region_poly],
    crs=crs,
)

# sanity check
print("Region polygon bounds:", region_gdf.total_bounds)
assert np.isfinite(region_gdf.total_bounds).all(), "❌ Region bounds contain non-finite values."

# ------------------------------------------------------------
# 2) Setup SFINCS region (works across versions)
# ------------------------------------------------------------
m.setup_region(region={"geom": region_gdf})
print("✅ setup_region(region={'geom': gdf}) worked")

# confirm HydroMT stored region properly
if "region" in getattr(m, "geoms", {}):
    print("Stored region bounds:", m.geoms["region"].total_bounds)
else:
    print("⚠️ m.geoms['region'] not found (not fatal, depends on HydroMT version).")

# ------------------------------------------------------------
# 3) ROBUST GRID FIX: Force SFINCS grid to match DEM grid exactly
#    (avoids setup_grid_from_region bugs + empty m.grid)
# ------------------------------------------------------------
grid = xr.Dataset(
    coords={
        "x": dem["x"].astype("float64"),
        "y": dem["y"].astype("float64"),
    }
)

# Attach CRS metadata + declare x/y as spatial dims (via rioxarray)
grid = grid.rio.write_crs(crs)
grid = grid.rio.set_spatial_dims(x_dim="x", y_dim="y")

# Assign to model (different HydroMT versions support different APIs)
if hasattr(m, "set_grid"):
    m.set_grid(grid)
    print("✅ m.set_grid(grid) worked")
else:
    m.grid = grid
    print("✅ m.grid = grid assigned directly")

# ------------------------------------------------------------
# 4) Print grid info safely (no hydromt.raster accessor assumptions)
# ------------------------------------------------------------
print("\n--- m.grid (after forcing DEM grid) ---")
print(m.grid)
print("Grid dims:", list(m.grid.dims))
print("Grid sizes:", dict(m.grid.sizes))

# Raster-like checks via rioxarray accessor
try:
    print("✅ Grid CRS:", m.grid.rio.crs)
    print("✅ Grid bounds:", m.grid.rio.bounds())
    # resolution from coords
    dx = float(np.abs(np.diff(m.grid["x"].values).mean())) if m.grid.sizes["x"] > 1 else np.nan
    dy = float(np.abs(np.diff(m.grid["y"].values).mean())) if m.grid.sizes["y"] > 1 else np.nan
    print("✅ Grid approx res (dx,dy):", (dx, dy))
except Exception as e:
    print("⚠️ Could not compute grid rio info:", repr(e))

print("\n✅ Region + DEM-aligned grid initialization complete.")


DEM bounds: [594250.0, 4517925.0, 597675.0, 4521200.0]
DEM CRS: EPSG:26918
DEM res: 25.0
Region polygon bounds: [ 594250. 4517925.  597675. 4521200.]
✅ setup_region(region={'geom': gdf}) worked
Stored region bounds: [ 594250. 4517925.  597675. 4521200.]
✅ m.set_grid(grid) worked

--- m.grid (after forcing DEM grid) ---
<xarray.Dataset> Size: 2kB
Dimensions:      (x: 137, y: 131)
Coordinates:
  * x            (x) float64 1kB 5.943e+05 5.943e+05 ... 5.976e+05 5.977e+05
  * y            (y) float64 1kB 4.521e+06 4.521e+06 ... 4.518e+06 4.518e+06
    band         int64 8B 1
    spatial_ref  int64 8B 0
Data variables:
    *empty*
Grid dims: ['x', 'y']
Grid sizes: {'x': 137, 'y': 131}
✅ Grid CRS: EPSG:26918
✅ Grid bounds: (594250.0, 4517925.0, 597675.0, 4521200.0)
✅ Grid approx res (dx,dy): (25.0, 25.0)

✅ Region + DEM-aligned grid initialization complete.


In [18]:
# ============================================================
# 5) Setup dep + ACTIVE mask (FIXED)
#    ✅ setup_mask_active must NOT use mask="dep" in your version
#    ✅ Instead, activate the whole bbox region geometry
# ============================================================

import geopandas as gpd
from shapely.geometry import box
import numpy as np

dep_path = str(paths_local["dep_dem"])
print("Using DEM:", dep_path)

# ---- 1) Setup dep (topobathy) ----
datasets_dep = [{"elevtn": dep_path, "name": "dem_meters"}]

m.setup_dep(
    datasets_dep=datasets_dep,
    interp_method="linear",
)

print("✅ dep written to model maps.")
print("Maps now include:", list(m.maps.keys()))

# ---- 2) Build an ACTIVE MASK from bbox geometry (robust + simple) ----
# Use DEM bounds (same CRS as model)
minx, miny, maxx, maxy = map(float, dem.rio.bounds())
bbox_poly = box(minx, miny, maxx, maxy)
bbox_gdf = gpd.GeoDataFrame({"name": ["active_bbox"]}, geometry=[bbox_poly], crs=dem.rio.crs)

assert np.isfinite(bbox_gdf.total_bounds).all(), "bbox bounds invalid"

# IMPORTANT: pass a GeoDataFrame (NOT a string like 'dep')
m.setup_mask_active(mask=bbox_gdf, reset_mask=True)

print("✅ Active mask created from bbox geometry.")
print("Mask shape:", m.mask.shape if hasattr(m, "mask") else "unknown")

# Quick sanity check
if hasattr(m, "mask"):
    print("Mask unique values:", np.unique(m.mask.values))




Using DEM: /tmp/sfincs_run_inputs_1mk6ynd6/dep_dem_utm25m_meters.tif
✅ dep written to model maps.
Maps now include: []
✅ Active mask created from bbox geometry.
Mask shape: (131, 137)
Mask unique values: [1]


In [19]:
# ============================================================
# 6) Setup Manning roughness (friction)
# ============================================================

import os

MANNING_NODATA = -9999.0

manning_fn = str(paths_local["manning"])
print("Using Manning raster:", manning_fn)

# hydromt_sfincs expects datasets_rgh as list of dicts with "manning"
datasets_rgh = [{"manning": manning_fn}]

m.setup_manning_roughness(
    datasets_rgh=datasets_rgh
)

print("✅ Manning roughness setup done.")
print("Maps now contain:", list(m.maps.keys()))


Using Manning raster: /tmp/sfincs_run_inputs_1mk6ynd6/manning_n_utm25m.tif
✅ Manning roughness setup done.
Maps now contain: []


In [21]:
# ============================================================
# 7) Setup infiltration from Curve Number raster (CN -> scsfile)
#    ✅ Robust to hydromt_sfincs version differences
# ============================================================

import inspect
import numpy as np
import rioxarray

CN_NODATA = -9999.0  # your convention
cn_fn = str(paths_local["curve_number"])

print("Using Curve Number raster:", cn_fn)

# --- Quick sanity check: file can be opened + has CRS ---
cn_da = rioxarray.open_rasterio(cn_fn, masked=False).squeeze()
print("CN dtype:", cn_da.dtype, "| shape:", cn_da.shape)
print("CN CRS:", cn_da.rio.crs, "| nodata:", cn_da.rio.nodata)

# --- Check method signature for your installed hydromt_sfincs ---
sig = inspect.signature(m.setup_cn_infiltration)
print("setup_cn_infiltration signature:", sig)

# --- Call correctly depending on version ---
kwargs = {}

# Some versions accept cn as keyword
if "cn" in sig.parameters:
    kwargs["cn"] = cn_fn
else:
    # Fallback: pass as positional argument
    kwargs = None

try:
    if kwargs is None:
        m.setup_cn_infiltration(cn_fn)
    else:
        m.setup_cn_infiltration(**kwargs)

    print("✅ CN infiltration setup done.")
except Exception as e:
    print("❌ setup_cn_infiltration failed:", repr(e))
    print("\nTrying fallback: constant infiltration (disable CN)…")

    # Optional fallback if CN method is incompatible
    # This is a *backup* path so your model can still run:
    # - user can set a uniform qinf (m/s) if needed
    # Example: 5 mm/hr ≈ 1.3889e-6 m/s
    qinf_ms = 1.3889e-6
    print(f"⚠️ Using fallback constant infiltration: qinf={qinf_ms:.3e} m/s")

    sig2 = inspect.signature(m.setup_constant_infiltration)
    print("setup_constant_infiltration signature:", sig2)

    if "qinf" in sig2.parameters:
        m.setup_constant_infiltration(qinf=qinf_ms)
    else:
        m.setup_constant_infiltration(qinf_ms)

    print("✅ Constant infiltration setup done instead.")

# --- Report maps now available ---
print("\nMaps now contain:", list(m.maps.keys()))


Using Curve Number raster: /tmp/sfincs_run_inputs_1mk6ynd6/curve_number_cn_utm25m.tif
CN dtype: float32 | shape: (131, 137)
CN CRS: EPSG:26918 | nodata: -9999.0
setup_cn_infiltration signature: (cn, antecedent_moisture='avg', reproj_method='med')
✅ CN infiltration setup done.

Maps now contain: []


In [27]:
# ============================================================
# 8) Locate + Validate precipitation forcing (SPATIALLY UNIFORM)
#    We use SFINCS "precipfile" text format directly (NOT NetCDF)
#
# Expected format (Deltares SFINCS precipfile):
#   - whitespace separated
#   - either:
#       (A) 2 columns:  <time> <precip>
#       (B) 1 column :  <precip>  (implicit timestep index)
#
# This cell finds the file, validates it, and sets:
#   precip_txt_local = "/path/to/sfincs.precip"
# ============================================================

from pathlib import Path
import numpy as np
import pandas as pd

# ------------------------------------------------------------
# 0) Find precip file robustly
# ------------------------------------------------------------
precip_txt_local = globals().get("precip_txt_local", None)

# A) if already specified manually elsewhere
if precip_txt_local is not None:
    precip_txt_local = str(precip_txt_local)

# B) try dictionary keys
if precip_txt_local is None and "paths_local" in globals() and isinstance(paths_local, dict):
    for k in ["precip", "rain", "rainfall", "sfincs_precip", "sfincs.precip", "precip_txt"]:
        if k in paths_local:
            precip_txt_local = str(paths_local[k])
            print(f"✅ Found precip file from paths_local['{k}'] -> {precip_txt_local}")
            break

# C) try autodetect inside cache_dir
if precip_txt_local is None:
    cache_dir = globals().get("cache_dir", None)
    if cache_dir is not None:
        cache_dir = Path(cache_dir)
        print("Searching cache_dir:", cache_dir)

        candidates = []
        # prefer classic precipfile names
        candidates += list(cache_dir.glob("sfincs.precip"))
        candidates += list(cache_dir.glob("*sfincs*precip*"))
        candidates += list(cache_dir.glob("*precip*"))
        candidates += list(cache_dir.glob("*rain*"))
        # also allow csv/txt/dat extensions
        candidates += list(cache_dir.glob("*.csv"))
        candidates += list(cache_dir.glob("*.txt"))
        candidates += list(cache_dir.glob("*.dat"))

        # keep only files
        candidates = [p for p in candidates if p.exists() and p.is_file()]

        # score: prefer exact "sfincs.precip", then names containing "precip"
        def score(p: Path):
            n = p.name.lower()
            return (
                0 if n == "sfincs.precip" else 1,
                0 if "precip" in n else 2,
                0 if "rain" in n else 3,
                len(n),
            )

        candidates = sorted(set(candidates), key=score)

        if len(candidates) > 0:
            precip_txt_local = str(candidates[0])
            print("✅ Auto-detected precip candidate:", precip_txt_local)
            if len(candidates) > 1:
                print("Other candidates (top 5):")
                for c in candidates[1:6]:
                    print(" -", c)
        else:
            raise FileNotFoundError(
                "❌ Could not auto-detect any precip text/CSV file in cache_dir.\n"
                "Fix: set precip_txt_local manually."
            )

if precip_txt_local is None:
    raise FileNotFoundError(
        "❌ Precip file not found.\n"
        "Fix: set precip_txt_local = '/path/to/sfincs.precip'"
    )

precip_txt_local = Path(precip_txt_local)
print("\nUsing precip forcing file (TEXT):", precip_txt_local)

if not precip_txt_local.exists():
    raise FileNotFoundError(f"❌ Missing precip file: {precip_txt_local}")

print("Exists ✅ | size bytes:", precip_txt_local.stat().st_size)

# ------------------------------------------------------------
# 1) Quick binary/text sanity check (must be text)
# ------------------------------------------------------------
with open(precip_txt_local, "rb") as f:
    head = f.read(64)

# if it looks like NetCDF/HDF, reject (you said no NetCDF)
if head[:3] == b"CDF" or head.startswith(b"\x89HDF\r\n\x1a\n"):
    raise RuntimeError(
        "❌ This precip file is actually a NetCDF/HDF binary file.\n"
        f"Path: {precip_txt_local}\n"
        "But we expect a TEXT precipfile (sfincs.precip)."
    )

# ------------------------------------------------------------
# 2) Parse as whitespace table (1 or 2 columns)
# ------------------------------------------------------------
df = pd.read_csv(
    precip_txt_local,
    sep=r"\s+",
    header=None,
    engine="python",
    comment="#",
)

if df.shape[1] == 0:
    raise RuntimeError("❌ Precip file parsed to 0 columns. File is empty or invalid.")

if df.shape[1] == 1:
    df.columns = ["precip"]
    df["time"] = np.arange(len(df), dtype="int64")
    df = df[["time", "precip"]]
    fmt = "1-column precipfile (implicit timestep index)"
elif df.shape[1] >= 2:
    df = df.iloc[:, :2].copy()
    df.columns = ["time", "precip"]
    fmt = "2-column precipfile (time + precip)"
else:
    raise RuntimeError("❌ Unexpected precip table shape.")

print("Parsed precip format:", fmt)
print("Rows:", len(df))

# ------------------------------------------------------------
# 3) Validate numeric precip
# ------------------------------------------------------------
df["precip"] = pd.to_numeric(df["precip"], errors="coerce")
n_bad = int(df["precip"].isna().sum())
if n_bad > 0:
    raise RuntimeError(f"❌ Precip column has {n_bad} NaNs (non-numeric values).")

pmin = float(df["precip"].min())
pmax = float(df["precip"].max())
print("Precip min/max:", pmin, pmax)

if pmax < 0:
    raise RuntimeError("❌ Precipitation is negative everywhere. Probably wrong units/sign.")

# optional: warn if huge values
if pmax > 500:
    print("⚠️ Warning: very large precip values (>500). Check units (mm/hr vs mm/day).")

# ------------------------------------------------------------
# 4) Validate time column (numeric or datetime-like string)
# ------------------------------------------------------------
tcol = df["time"].iloc[:10].astype(str).tolist()

# attempt datetime parse if it looks like datetime strings
looks_datetime = any(("-" in s or "T" in s or ":" in s) for s in tcol)

if looks_datetime:
    t = pd.to_datetime(df["time"], errors="coerce")
    if t.isna().any():
        raise RuntimeError("❌ time column looked datetime-like but datetime parsing failed.")
    print("Time parsed as datetime ✅")
    print("time start:", t.iloc[0])
    print("time end  :", t.iloc[-1])
else:
    # numeric time
    tnum = pd.to_numeric(df["time"], errors="coerce")
    if tnum.isna().any():
        raise RuntimeError("❌ time column is neither valid datetime nor numeric.")
    tnum = tnum.astype("int64").values
    print("Time parsed as numeric ✅")
    print("time min/max:", int(tnum.min()), int(tnum.max()))

    if len(tnum) >= 2:
        dt = np.unique(np.diff(tnum[:min(len(tnum), 1000)]))
        if len(dt) > 10:
            print("⚠️ time step looks irregular (many dt values):", dt[:10])
        else:
            print("✅ dt (first unique values):", dt[:10])

# ------------------------------------------------------------
# 5) Print preview
# ------------------------------------------------------------
print("\n--- First 5 lines (parsed) ---")
print(df.head())

print("\n✅ Precip forcing validated as spatially-uniform SFINCS precipfile text.")
print("✅ Use this path later as precip_txt_local =", str(precip_txt_local))


# ============================================================
# 8b) Fix precip CSV time axis (convert numeric -> datetime in 2025)
# ============================================================

import numpy as np
import pandas as pd

precip_local = globals().get("precip_local", None)
if precip_local is None:
    raise RuntimeError("❌ precip_local not set. Run Cell 8 first.")

print("Using precip CSV:", precip_local)

dfp = pd.read_csv(
    precip_local,
    sep=r"\s+",
    header=None,
    names=["time_raw", "rain"],
)

print("\n--- Precip CSV preview ---")
print(dfp.head())
print(dfp.tail())
print("\nRaw time dtype:", dfp["time_raw"].dtype)

# ---- Case A: time is already ISO-like strings
if dfp["time_raw"].dtype == object:
    t = pd.to_datetime(dfp["time_raw"], errors="raise")
    dfp["time"] = t

# ---- Case B: time is numeric (likely minutes since start)
else:
    tnum = dfp["time_raw"].to_numpy()

    # Heuristic: interpret as minutes since simulation start
    # This is consistent with your FEWS wind convention, and typical SFINCS forcing.
    # If your teammate used seconds instead of minutes, we detect it below.
    dt_guess_minutes = np.median(np.diff(tnum[:min(len(tnum), 1000)]))
    print("\nMedian delta between time rows:", dt_guess_minutes)

    # Detect seconds vs minutes
    # if timestep is ~60 -> seconds
    # if timestep is ~1  -> minutes (hourly data would be 60 minutes)
    if 50 <= dt_guess_minutes <= 70:
        units = "seconds"
        base = pd.Timestamp("2025-01-01 00:00:00")
        dfp["time"] = base + pd.to_timedelta(tnum, unit="s")
    else:
        units = "minutes"
        base = pd.Timestamp("2025-01-01 00:00:00")
        dfp["time"] = base + pd.to_timedelta(tnum, unit="m")

    print("✅ Interpreting precip time as", units, "since", base)

print("\n✅ Parsed precip datetime range:")
print("start:", dfp["time"].iloc[0])
print("end  :", dfp["time"].iloc[-1])

# Save back into globals for downstream cells
precip_df = dfp



Searching cache_dir: /tmp/sfincs_run_inputs_1mk6ynd6
✅ Auto-detected precip candidate: /tmp/sfincs_run_inputs_1mk6ynd6/sfincs_precip.nc

Using precip forcing file (TEXT): /tmp/sfincs_run_inputs_1mk6ynd6/sfincs_precip.nc
Exists ✅ | size bytes: 197868
Parsed precip format: 2-column precipfile (time + precip)
Rows: 8038
Precip min/max: 0.0 28.504
Time parsed as numeric ✅
time min/max: 0 28940400
✅ dt (first unique values): [3600 7200]

--- First 5 lines (parsed) ---
      time  precip
0      0.0   0.000
1   3600.0   0.017
2   7200.0   0.170
3  10800.0   8.092
4  14400.0   0.781

✅ Precip forcing validated as spatially-uniform SFINCS precipfile text.
✅ Use this path later as precip_txt_local = /tmp/sfincs_run_inputs_1mk6ynd6/sfincs_precip.nc


In [32]:
# ============================================================
# 9) Attach wind forcing (ROBUST: time-overlap safe + rename + CRS)
# ============================================================

from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr

# ------------------------------
# 0) paths
# ------------------------------
wind_local = globals().get("wind_local", "/tmp/sfincs_run_inputs_1mk6ynd6/netamuamvfile.nc")
precip_local = globals().get("precip_local", None)  # should exist from Cell 8

print("Using wind forcing file:", wind_local)
if precip_local is not None:
    print("Using precip forcing file:", precip_local)
else:
    print("⚠️ precip_local not found in globals(); will attach wind without time slicing (may fail).")

if not Path(wind_local).exists():
    raise FileNotFoundError(f"❌ Wind file not found: {wind_local}")

# ------------------------------
# 1) open wind file + rename
# ------------------------------
ds_w = xr.open_dataset(wind_local, engine="netcdf4")

rename_map = {}
if "amu" in ds_w:
    rename_map["amu"] = "wind10_u"
if "amv" in ds_w:
    rename_map["amv"] = "wind10_v"

ds_w = ds_w.rename(rename_map)

if "wind10_u" not in ds_w or "wind10_v" not in ds_w:
    raise RuntimeError(
        "❌ Wind dataset must contain wind10_u and wind10_v.\n"
        f"Vars found: {list(ds_w.data_vars)}"
    )

# ensure dims
needed = {"time", "y", "x"}
if not needed.issubset(ds_w.dims):
    raise RuntimeError(f"❌ Wind dims must include {needed}, got {dict(ds_w.sizes)}")

# float32
ds_w["wind10_u"] = ds_w["wind10_u"].astype("float32")
ds_w["wind10_v"] = ds_w["wind10_v"].astype("float32")

print("\n--- Wind dataset (after rename) ---")
print(ds_w)

# ------------------------------
# 2) attach HydroMT raster metadata (IMPORTANT)
# ------------------------------
# use DEM CRS if available
if "dem" in globals() and hasattr(dem, "rio") and dem.rio.crs is not None:
    crs_guess = str(dem.rio.crs)
else:
    crs_guess = ds_w.attrs.get("crs", "EPSG:26918")

ds_w.attrs["crs"] = crs_guess

# make x/y increasing (helps slicer)
if np.any(np.diff(ds_w["x"].values) < 0):
    ds_w = ds_w.isel(x=slice(None, None, -1))
if np.any(np.diff(ds_w["y"].values) < 0):
    ds_w = ds_w.isel(y=slice(None, None, -1))

# attach hydromt raster accessor (inplace!)
ds_w.raster.set_spatial_dims(x_dim="x", y_dim="y")
ds_w.raster.set_crs(crs_guess)

print("\n✅ Wind CRS via raster:", ds_w.raster.crs)

# ------------------------------
# 3) determine precip time range (CSV uniform rainfall)
# ------------------------------
t_precip0 = None
t_precip1 = None

if precip_local is not None and Path(precip_local).exists():
    # your precip file is a 2-column space-separated CSV:
    # time precipitation
    # with no headers
    dfp = pd.read_csv(
        precip_local,
        sep=r"\s+",
        header=None,
        names=["time", "rain"],
    )

    # Try parsing time column to datetime
    # (This works if the first column is ISO time strings like 2020-01-01T00:00:00)
    try:
        t = pd.to_datetime(dfp["time"])
        t_precip0 = np.datetime64(t.iloc[0].to_datetime64())
        t_precip1 = np.datetime64(t.iloc[-1].to_datetime64())
        print("\n✅ Precip time range:", t_precip0, "->", t_precip1)
    except Exception as e:
        print("\n⚠️ Could not parse precip time column as datetime.")
        print("   Sample first rows:\n", dfp.head())
        print("   Error:", type(e).__name__, "-", e)

# ------------------------------
# 4) slice wind to precip overlap if possible
# ------------------------------
t_w0 = ds_w["time"].values[0]
t_w1 = ds_w["time"].values[-1]
print("\n✅ Wind time range:", t_w0, "->", t_w1)

if (t_precip0 is not None) and (t_precip1 is not None):
    # overlap window
    t0 = max(t_precip0, t_w0)
    t1 = min(t_precip1, t_w1)

    if t1 < t0:
        raise RuntimeError(
            "❌ NO TIME OVERLAP between precip CSV and wind NetCDF.\n\n"
            f"Precip: {t_precip0} -> {t_precip1}\n"
            f"Wind  : {t_w0} -> {t_w1}\n\n"
            "Fix: regenerate the wind forcing for the same year as precipitation "
            "(or regenerate precipitation for the wind year)."
        )

    ds_w = ds_w.sel(time=slice(t0, t1))
    print("\n✅ Sliced wind to overlap window:", ds_w.time.values[0], "->", ds_w.time.values[-1])
else:
    print("\n⚠️ Precip time range unknown; skipping wind time slicing.")

# ------------------------------
# 5) set model time window (IMPORTANT so HydroMT doesn't slice wind to None)
# ------------------------------
# try to set start/stop in model config if available
if hasattr(m, "config") and isinstance(m.config, dict) and ds_w.sizes["time"] > 0:
    t0 = pd.to_datetime(ds_w.time.values[0]).strftime("%Y%m%d %H%M%S")
    t1 = pd.to_datetime(ds_w.time.values[-1]).strftime("%Y%m%d %H%M%S")

    # these are common SFINCS config keys (harmless if unused)
    m.config["tstart"] = t0
    m.config["tstop"]  = t1
    print("\n✅ Set model config time:")
    print("   tstart =", m.config["tstart"])
    print("   tstop  =", m.config["tstop"])
else:
    print("\n⚠️ Could not set model config time (m.config not available).")

# ------------------------------
# 6) attach wind forcing
# ------------------------------
m.setup_wind_forcing_from_grid(wind=ds_w)
print("\n✅ Wind forcing attached successfully.")
print("Forcing keys now:", list(getattr(m, "forcing", {}).keys()))



Using wind forcing file: /tmp/sfincs_run_inputs_1mk6ynd6/netamuamvfile.nc
Using precip forcing file: /tmp/sfincs_run_inputs_1mk6ynd6/sfincs_precip.nc

--- Wind dataset (after rename) ---
<xarray.Dataset> Size: 211kB
Dimensions:   (time: 8784, y: 1, x: 2)
Coordinates:
  * time      (time) datetime64[ns] 70kB 2020-01-01 ... 2020-12-31T23:00:00
  * y         (y) float64 8B 4.52e+06
  * x         (x) float64 16B 5.946e+05 5.975e+05
Data variables:
    wind10_u  (time, y, x) float32 70kB -2.123 -2.06 -2.541 ... -7.051 -6.989
    wind10_v  (time, y, x) float32 70kB -6.651 -6.588 -6.98 ... -5.794 -5.856
Attributes:
    crs:      EPSG:26918

✅ Wind CRS via raster: EPSG:26918

✅ Precip time range: 1970-01-01T00:00:00.000000000 -> 1970-01-01T00:00:00.028940400

✅ Wind time range: 2020-01-01T00:00:00.000000000 -> 2020-12-31T23:00:00.000000000


RuntimeError: ❌ NO TIME OVERLAP between precip CSV and wind NetCDF.

Precip: 1970-01-01T00:00:00.000000000 -> 1970-01-01T00:00:00.028940400
Wind  : 2020-01-01T00:00:00.000000000 -> 2020-12-31T23:00:00.000000000

Fix: regenerate the wind forcing for the same year as precipitation (or regenerate precipitation for the wind year).